In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [35]:
df = pd.read_csv('./data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')
    
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors = 'coerce')
df['totalcharges'] = df['totalcharges'].fillna(0)

df['churn'] = (df['churn'] == 'yes').astype(int)

In [36]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [37]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [38]:
def train(df_train, y_train, C = 1.0):
    dicts = df_train[categorical + numerical].to_dict(orient = 'records')
    
    dv = DictVectorizer(sparse = False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C = C, max_iter = 1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [39]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient = 'records')
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [40]:
C = 1.0
n_splits = 5

In [42]:
kfold = KFold(n_splits = n_splits, shuffle = True, random_state = 1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    
    y_train = df_train['churn'].values
    y_val = df_val['churn'].values
        
    dv, model = train(df_train, y_train, C = C)
    y_pred = predict(df_val, dv, model)
    
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
    
print('C = %s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

C = 1.0 0.842 +- 0.007


In [43]:
scores

[0.8445023588793557,
 0.8451508906105152,
 0.833289183222958,
 0.8347449159045852,
 0.8517500235323648]

In [44]:
dv, model = train(df_full_train, df_full_train['churn'].values, C = C)
y_pred = predict(df_test, dv, model)

y_test = df_test['churn'].values
auc = roc_auc_score(y_test, y_pred)
auc

0.8583734169672939

### Save the model

In [45]:
import pickle

In [46]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [47]:
# ### 1 way to save file
# # w for write, b for binary -> save the file as binary
# f_out = open(output_file, 'wb')
# # save dv and model
# pickle.dump((dv, model), f_out)
# # close the file
# f_out.close()

# --------------------------------------------------------------

### Another way to save file without close()
# w for write, b for binary -> save the file as binary
with open(output_file, 'wb') as f_out:
    # do stuff
    
    # save dv and model
    pickle.dump((dv, model), f_out)
    
# do other stuff

### Load the model
(restart the kernel, assuming that you are working on different machine)

In [48]:
# Need to have sklearn in your machine otherwise the dv, model cannot be loaded
import pickle

In [49]:
model_file = 'model_C=1.0.bin'

In [50]:
with open(model_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [51]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [59]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [60]:
X = dv.transform([customer])

In [61]:
model.predict_proba(X)[0, 1]

0.6278846053082351